<a href="https://colab.research.google.com/github/huankoh/PSICHIC/blob/main/PSICHIC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $\text{PSICHIC}_{\mathbb{V1.0}} \text{Virtual Screening Platform on Google Colab}$

| [Paper](X) | [Github Repo](X) | [Run in Colab](X) (click `Runtime` → `Run all (Ctrl+F9)` |

## PSICHIC Features

- **Quick Screening**: Up to 100K compounds in an hour.
- **Deep Analysis**: Uncover molecular insights with PSICHIC's unique algorithms.
- **More Tools**: Pharmacophore and targeted mutagenesis analysis.

## Input Needed

- **Only Sequence Data**: Protein Sequence + Ligand SMILES pairs.

## Why PSICHIC?

- **Fast**: Save time on initial screenings.
- **Smart**: Make data-driven decisions.
- **Efficient**: No special hardware needed.

Start exploring. Your next discovery is just clicks away!

In [ ]:
%%time
#@title Setting up PSICHIC Online Platform (~5min 30s)
import os, time

# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

os.system(f"pip install torch-scatter -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html")
os.system(f"pip install torch-sparse -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html")
os.system(f"pip install torch-cluster -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html")
os.system(f"pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{TORCH}+{CUDA}.html")
os.system(f"pip install torch-geometric")

os.system("pip install rdkit")
os.system("pip install biopython")
os.system("pip install biopandas")
os.system("pip install py3Dmol")
os.system("pip install timeout_decorator")
os.system("pip install cairosvg")
os.system("pip install umap-learn")
os.system("pip install plotly")
os.system("pip install --upgrade plotly")
os.system("pip install mplcursors")

os.system("pip install lifelines")
os.system("pip install reprint")
# install PSICHIC
# !git clone https://github.com/huankoh/PSICHIC.git
version = "1"
model_name = "esmfold_v0.model" if version == "0" else "esmfold.model"

if not os.path.isfile(model_name):
  # download esmfold params
  os.system("apt-get install aria2 -qq")
  os.system(f"aria2c -q -x 16 https://colabfold.steineggerlab.workers.dev/esm/{model_name} &")

  if not os.path.isfile("finished_install"):
    print("installing esmfold...")
    # install libs
    os.system("pip install -q omegaconf pytorch_lightning biopython ml_collections einops py3Dmol")
    os.system("pip install -q git+https://github.com/NVIDIA/dllogger.git")

    # install openfold
    commit = "6908936b68ae89f67755240e2f588c09ec31d4c8"
    os.system(f"pip install -q git+https://github.com/aqlaboratory/openfold.git@{commit}")

    # install esmfold
    os.system(f"pip install -q git+https://github.com/sokrypton/esm.git")
    os.system("touch finished_install")




  # wait for Params to finish downloading...
  while not os.path.isfile(model_name):
    time.sleep(5)
  if os.path.isfile(f"{model_name}.aria2"):
    print("downloading params...")
  while os.path.isfile(f"{model_name}.aria2"):
    time.sleep(5)

import torch
import os
import pandas as pd
from IPython.display import SVG
from rdkit import Chem
import numpy as np


In [ ]:
#@title 1. Importing functions
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
from ipywidgets import Button, Layout, jslink, IntText, IntSlider
import io
import IPython
from ipywidgets import GridspecLayout
from ipywidgets import Output

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import Bio.PDB as PDB

import py3Dmol


from rdkit.Chem.Draw import SimilarityMaps
from rdkit.Chem import Draw
import pickle

from string import ascii_uppercase, ascii_lowercase
import hashlib, re, os
import numpy as np
import torch
from jax.tree_util import tree_map
import matplotlib.pyplot as plt
from scipy.special import softmax
import gc
import warnings
from rdkit.Chem import AllChem

def read_molecule(molecule_file, sanitize=False, calc_charges=False, remove_hs=True):
    if molecule_file.endswith('.mol2'):
        mol = Chem.MolFromMol2File(molecule_file, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.sdf'):
        supplier = Chem.SDMolSupplier(molecule_file, sanitize=False, removeHs=False)
        mol = supplier[0]
    elif molecule_file.endswith('.pdbqt'):
        with open(molecule_file) as file:
            pdbqt_data = file.readlines()
        pdb_block = ''
        for line in pdbqt_data:
            pdb_block += '{}\n'.format(line[:66])
        mol = Chem.MolFromPDBBlock(pdb_block, sanitize=False, removeHs=False)
    elif molecule_file.endswith('.pdb'):
        mol = Chem.MolFromPDBFile(molecule_file, sanitize=False, removeHs=False)
    else:
        raise ValueError('Expect the format of the molecule_file to be '
                         'one of .mol2, .sdf, .pdbqt and .pdb, got {}'.format(molecule_file))

    try:
        if sanitize or calc_charges:
            Chem.SanitizeMol(mol)

        if calc_charges:
            # Compute Gasteiger charges on the molecule.
            try:
                AllChem.ComputeGasteigerCharges(mol)
            except:
                warnings.warn('Unable to compute charges for the molecule.')

        if remove_hs:
            mol = Chem.RemoveHs(mol, sanitize=sanitize)

    except Exception as e:
        # print(e)
        return None

    return mol


def read_mol(mol_path, remove_hs=False):
    lig = None
    if '.sdf' in mol_path:
        lig = read_molecule(mol_path, remove_hs=remove_hs, sanitize=True)
    elif '.mol2' in mol_path:  # read mol2 file if sdf file cannot be sanitized
        lig = read_molecule(mol_path, remove_hs=remove_hs, sanitize=True)
    if lig is None:
        raise Exception("Fail to read ligand path")

    return lig



show_sidechains=False

mapper = plt.get_cmap('Reds', )
color_map = {}


one_to_three = {"A" : "ALA",
              "C" : "CYS",
              "D" : "ASP",
              "E" : "GLU",
              "F" : "PHE",
              "G" : "GLY",
              "H" : "HIS",
              "I" : "ILE",
              "K" : "LYS",
              "L" : "LEU",
              "M" : "MET",
              "N" : "ASN",
              "P" : "PRO",
              "Q" : "GLN",
              "R" : "ARG",
              "S" : "SER",
              "T" : "THR",
              "V" : "VAL",
              "W" : "TRP",
              "Y" : "TYR",
              "B" : "ASX",
              "Z" : "GLX",
              "X" : "UNK",
              "*" : " * "}

three_to_one = {}
for _key, _value in one_to_three.items():
    three_to_one[_value] = _key
three_to_one["SEC"] = "C"
three_to_one["MSE"] = "M"
def overwrite_b_factors(structure,
                        custom_bfactors: np.ndarray, res_symbols: list,
                        save_path=None) -> str:

    curr_resid = ('', '', '')
    idx = -1
    a_list = []
    for i, chain in enumerate(structure):
        for res_idx, residue in enumerate(chain):
            if residue.get_resname() == 'HOH':
                for atom in residue:
                    atom.bfactor = 0

            c_alpha, n, c = None, None, None
            for atom in residue:
                if atom.name == 'CA':
                    c_alpha = list(atom.get_vector())
                if atom.name == 'N':
                    n = list(atom.get_vector())
                if atom.name == 'C':
                    c = list(atom.get_vector())
            if c_alpha != None and n != None and c != None:
                idx += 1

                for atom in residue:
                    ##### ensure visualization SEQ follows pre-processed SEQ
                    try:
                        pdb_res_symbol = three_to_one[residue.get_resname()]
                    except Exception as e:
                        pdb_res_symbol = 'X'
                        # print("encountered unknown AA: ", residue.get_resname(), ' in the complex. Replacing it with a dash X.')

                    if res_symbols[idx] != pdb_res_symbol:
                        raise Exception('Result sequence symbol {} does not match with PDB sequence symbol {}'.format(res_symbols[idx],pdb_res_symbol))

                    atom.bfactor = custom_bfactors[idx]
            else:
                for atom in residue:
                    atom.bfactor = 0
    if save_path is None:
        path_out = 'temporary.txt'
    else:
        path_out = save_path
    new_pdb = io.StringIO()
    pdb_io = PDB.PDBIO()
    pdb_io.set_structure(structure)
    pdb_io.save(path_out)

    with open(path_out,'r') as f:
        out = f.read()

    return out

for v in range(100+1):
    rgba = mapper(v/100)
    color_map[v/100] = rgb2hex(rgba)

def DrawDocking(protein,ligand=None,
                show_protein_surface=True, show_protein_mainchain=False,show_protein_sidechain=False):
    viewer = py3Dmol.view(width=800,height=800)
    viewer.addModelsAsFrames(protein)

    if show_protein_sidechain:
        BB = ['C','O','N']
        viewer.addStyle({'and':[{'resn':["GLY","PRO"],'invert':True},{'atom':BB,'invert':True}]},
                    {'stick':{'colorscheme': {'prop': 'b', 'map': color_map},'radius':0.3}})
        viewer.addStyle({'and':[{'resn':"GLY"},{'atom':'CA'}]},
                    {'sphere':{'colorscheme': {'prop': 'b', 'map': color_map},'radius':0.3}})
        viewer.addStyle({'and':[{'resn':"PRO"},{'atom':['C','O'],'invert':True}]},
                    {'stick':{'colorscheme': {'prop': 'b', 'map': color_map},'radius':0.3}})

    if show_protein_mainchain:
        BB = ['C','O','N','CA']
        viewer.addModelsAsFrames
        viewer.addStyle({'atom':BB},{'stick':{'colorscheme': {'prop': 'b', 'map': color_map},'radius':0.3}})

    if show_protein_surface:
        BB = ['C','O','N','CA']
        prot = {'resn': ["DMS", "UNL", "SO4", "LIG", "HOH", "Cl"], 'invert': 1}  #define prot as all except list
        style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}
        viewer.addSurface(py3Dmol.VDW,{'opacity':0.8, 'colorscheme': {'prop': 'b', 'map': color_map}}, prot)

    if ligand is not None:
        viewer.addModel(ligand)
        MyLig = {'resn':'UNL'}
        viewer.setStyle(MyLig,{'stick':{'colorscheme' : 'greenCarbon'}})
        viewer.zoomTo(MyLig)
    else:
        viewer.zoomTo()
    return viewer

def alphafold_plot(to_visualize_pdb):
    view = py3Dmol.view(width=800, height=600)
    view.addModelsAsFrames(to_visualize_pdb)
    style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}
    if show_sidechains:
        style['stick'] = {}
    view.setStyle({'model': -1}, style)
    view.zoomTo()
    return view

def generate_ESM_structure(jobname, sequence):
    def get_hash(x): return hashlib.sha1(x.encode()).hexdigest()
    alphabet_list = list(ascii_uppercase+ascii_lowercase)

    jobname = re.sub(r'\W+', '', jobname)[:50]

    sequence = re.sub("[^A-Z:]", "", sequence.replace("/",":").upper())
    sequence = re.sub(":+",":",sequence)
    sequence = re.sub("^[:]+","",sequence)
    sequence = re.sub("[:]+$","",sequence)
    copies = 1
    sequence = ":".join([sequence] * copies)
    num_recycles = 3
    chain_linker = 25

    ID = jobname+"_"+get_hash(sequence)[:5]
    seqs = sequence.split(":")
    lengths = [len(s) for s in seqs]
    length = sum(lengths)
    print("length",length)

    u_seqs = list(set(seqs))
    if len(seqs) == 1: mode = "mono"
    elif len(u_seqs) == 1: mode = "homo"
    else: mode = "hetero"

    if "model" not in dir() or model_name != model_name_:
        if "model" in dir():
            # delete old model from memory
            del model
            gc.collect()
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        model = torch.load(model_name)
        model.eval().cuda().requires_grad_(False)
        model_name_ = model_name

    # optimized for Tesla T4
    if length > 700:
        model.set_chunk_size(64)
    else:
        model.set_chunk_size(128)

    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    output = model.infer(sequence,
                        num_recycles=num_recycles,
                        chain_linker="X"*chain_linker,
                        residue_index_offset=512)

    pdb_str = model.output_to_pdb(output)[0]

    del model
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    return pdb_str


def plot_visual(protein_csv, ligand_pickle,  protein_path=None, ligand_path=None, save_id='',
                residue_color_type='Original', show_protein_surface=True,
                show_protein_mainchain=False,show_protein_sidechain=False):
    protein_df = pd.read_csv(protein_csv)
    if residue_color_type == 'Original':
        residue_scores = protein_df['PSICHIC_Residue_Score'].round(2).clip(0.01,0.99).tolist()
    else:
        residue_scores = protein_df['PSICHIC_Residue_Percentile'].round(2).clip(0.01,0.99).tolist()

    if protein_path:
        parser = PDB.PDBParser(QUIET=True)
        structure = parser.get_structure('', protein_path)[0]
        protein_pdb_block = overwrite_b_factors(structure, residue_scores, protein_df['Residue_Type'].tolist())
    else:
        print('No PDB detected - generating in silico structure from ESM')
        pdbstr = generate_ESM_structure(save_id + '_ESM.pdb', ''.join(protein_df['Residue_Type'].tolist()))
        torch.cuda.empty_cache()

        parser = PDB.PDBParser(QUIET=True)
        handle = io.StringIO(pdbstr)
        structure = parser.get_structure('', handle)[0]
        protein_pdb_block = overwrite_b_factors(structure, residue_scores, protein_df['Residue_Type'].tolist())


    ligand_pdb_block = None
    if ligand_path:
        ligand = read_mol(ligand_path)
        ligand_pdb_block = Chem.MolToPDBBlock(ligand)


    view = DrawDocking(protein_pdb_block, ligand_pdb_block,
                       show_protein_surface=show_protein_surface,
                        show_protein_mainchain=show_protein_mainchain,
                        show_protein_sidechain=show_protein_sidechain)
    grid = GridspecLayout(1,2, layout=Layout(width='80%', height='80%'))
    out = Output()
    with out:
        view.show()
    grid[0, 0] = out

    ## atom scores
    with open(ligand_pickle,'rb') as f:
        mol = pickle.load(f)

    ligand_contribs = []
    for idx, atom in enumerate(mol.GetAtoms()):
        psichic_atom_score = atom.GetProp("PSICHIC_Atom_Score")
        atom.SetProp('score',psichic_atom_score)
        ligand_contribs.append(float(psichic_atom_score.replace('[','').replace(']','')))


    d1 = Draw.MolDraw2DSVG(500, 500)
    def make_darker(color, scale):
        r, g, b, a = color
        return (r * scale, g * scale, b * scale, a)

    start_color = mcolors.hex2color('#40d0e0')
    middle_color = mcolors.hex2color('white')
    end_color = mcolors.hex2color('#40d0e0')
    cmap = mcolors.LinearSegmentedColormap.from_list(
        'custom_cmap', [start_color, middle_color, end_color])
    darker_first_color = make_darker(cmap(0), 0.8)
    darker_last_color = make_darker(cmap(1), 0.8)

    colorMap = LinearSegmentedColormap.from_list('Wistia', [darker_first_color, (1.0, 1.0, 1.0), darker_last_color], N=255)
    SimilarityMaps.GetSimilarityMapFromWeights(mol, ligand_contribs, draw2d=d1, contourLines=10, colorMap=colorMap)
    d1.FinishDrawing()
    svg1 = SVG(d1.GetDrawingText())

    out = Output()
    with out:
        IPython.display.display(svg1)
    grid[0,1] = out
    IPython.display.display(grid)
    return protein_pdb_block, ligand_pdb_block

# PSICHIC Sequence-based Library Screening

- User-Friendly: Just a CSV file with ``'Protein`` and ``Ligand`` columns needed. It is desriable to also include an ``ID`` column to refer each unique pair during your analysis at later stage. If ``ID`` is missing, we will include an ID based on the row number (PAIR_1, ..., PAIR_N).

Your CSV should look something like this:

| ID | Protein | Ligand |
|:----------:|:----------:|:----------:|
| MIPS1| ATCGATCG....  | C1CCCCC1  |
| MIPS2| GCTAGCTA....  | O=C(C)Oc1ccccc1C(=O)O |
|...|...| ...|
|MIPS100K|TACGTACG | CCO |

- Fast & Accurate: Get reliable results quickly.
- Streamlined Workflow: All-in-one solution for protein-ligand interaction
analysis.

In [ ]:
#@title 1. Upload your screening csv file here ⬇️

#@markdown Running this code block, will prompt you to upload a csv file for virtual screening. If you just want to try PSICHIC, we have a file for PSICHIC demo, just click 'Cancel upload'.

#@markdown Specify the name of the result folder below (job_id):
jobname = "PSICHIC_RESULT" #@param {type:"string"}

import io
import pandas as pd
from google.colab import files


try:
    uploaded = files.upload()
    filename=[key for key in uploaded.keys()][0]
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
except:
    print('Fail to read any uploaded files. Using our demo file from PDB v2020 test data instead..')
    filename = 'dataset/pdb2020/test.csv'

In [ ]:
#@title 2. Run screening using PSICHIC
#@markdown By default, we will use the PSICHIC model trained on PDBBind v2020. This is the model that is most comprehensively evaluated (including its interpretability). You can also choose the PSICHIC-MultiTask model where we pre-trained it on Large-scale Interaction Database. For best performance, we suggest finetuning it first before using it as described in the manuscript and Github Repository.
PSICHIC_parameters = "Trained on PDBBind v2020" #@param ["Trained on PDBBind v2020", "Pre-trained on Large-scale Interaction Database"]
#@markdown By default, we will also save the interpretation from PSICHIC. If you only want the prediction output without further analysis, you can turn this off. However, in this case, you will not be able to use the rest of the notebook for analysis.
Save_PSICHIC_Interpretation = True #@param {type:"boolean"}
#@markdown By default, we use batch size of 16. In most cases, this is a good default - You may need to decrease it if the protein sequence length is very long.
batch_size = 16 #@param {type:"integer"}
if PSICHIC_parameters == 'Trained on PDBBind v2020':
    trained_model_path = 'trained_weights/PDBv2020_PSICHIC'
elif PSICHIC_parameters == 'Pre-trained on Large-scale Interaction Database':
    trained_model_path = 'trained_weights/multitask_PSICHIC'

screenfile = filename
result_path = jobname
#@markdown Three substep you will observe is (1) Initialising Protein, (2) Initialising Ligand and (3) Start Screening.

import json
import pandas as pd
import torch
import numpy as np
import os
import random
# Utils
from utils.utils import DataLoader, virtual_screening
from utils.dataset import *  # data
from utils.trainer import Trainer
from utils.metrics import *
# Preprocessing
from utils import protein_init, ligand_init
# Model
from models.net import net
# Config
from tqdm import tqdm

device = 'cuda:0'
with open(os.path.join(trained_model_path,'config.json'),'r') as f:
    config = json.load(f)

print("Screening the csv file: {}".format(screenfile))
# device
device = torch.device(device)


if not os.path.exists(result_path):
    os.makedirs(result_path)

degree_dict = torch.load(os.path.join(trained_model_path,'degree.pt'))
param_dict = os.path.join(trained_model_path,'model.pt')
mol_deg, prot_deg = degree_dict['ligand_deg'],degree_dict['protein_deg']

model = net(mol_deg, prot_deg,
            # MOLECULE
            mol_in_channels=config['params']['mol_in_channels'],  prot_in_channels=config['params']['prot_in_channels'],
            prot_evo_channels=config['params']['prot_evo_channels'],
            hidden_channels=config['params']['hidden_channels'], pre_layers=config['params']['pre_layers'],
            post_layers=config['params']['post_layers'],aggregators=config['params']['aggregators'],
            scalers=config['params']['scalers'],total_layer=config['params']['total_layer'],
            K = config['params']['K'],heads=config['params']['heads'],
            dropout=config['params']['dropout'],
            dropout_attn_score=config['params']['dropout_attn_score'],
            # output
            regression_head=config['tasks']['regression_task'],
            classification_head=config['tasks']['classification_task'] ,
            multiclassification_head=config['tasks']['mclassification_task'],
            device=device).to(device)
model.reset_parameters()
model.load_state_dict(torch.load(param_dict,map_location=device))


screen_df = pd.read_csv(os.path.join(screenfile))
protein_seqs = screen_df['Protein'].unique().tolist()
print('Initialising protein sequence to Protein Graph')
protein_dict = protein_init(protein_seqs)
ligand_smiles = screen_df['Ligand'].unique().tolist()
print('Initialising ligand SMILES to Ligand Graph')
ligand_dict = ligand_init(ligand_smiles)
torch.cuda.empty_cache()
## drop any invalid smiles
screen_df = screen_df[screen_df['Ligand'].isin(list(ligand_dict.keys()))].reset_index(drop=True)
screen_dataset = ProteinMoleculeDataset(screen_df, ligand_dict, protein_dict, device=device)
screen_loader = DataLoader(screen_dataset, batch_size=batch_size, shuffle=False,
                            follow_batch=['mol_x', 'clique_x', 'prot_node_aa'])

print("Screening starts now!")
screen_df = virtual_screening(screen_df, model, screen_loader,
                 result_path=os.path.join(result_path, "interpretation_result"), save_interpret=Save_PSICHIC_Interpretation,
                 ligand_dict=ligand_dict, device=device)

screen_df.to_csv(os.path.join(result_path,'screening.csv'),index=False)
print('Screening completed and saved to {}'.format(result_path))

In [ ]:
#@title 3. OPTIONAL - Download Screening Results
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

#@title Do you agree? { run: "auto" }
download_result = True #@param {type:"boolean"}

if download_result:
    os.system(f"zip -r {jobname}.result.zip {jobname}" )
    files.download(f"{jobname}.result.zip")

# PSICHIC Interactive Platform for Drug Discovery

This interactive UMAP scatter plot visualizes compounds based on their binding affinity and the probability of being an agonist, antagonist, or non-binder. This is a powerful tool for drug discovery, enabling you to quickly identify potential drug candidates and understand the chemical space.


#### Visual Encoding:

- **Size and Color**: The size of each point corresponds to its binding affinity. Larger points (redder color) indicate higher binding affinity.
- **Shape**: Points take different shapes based on whether they are most likely to be agonists, antagonists, or non-binders.

#### How to Explore:

1. **Hover Over Points**: Move your cursor over individual points to reveal their unique IDs, binding affinity, and agonist/antagonist/non-binder probability.
2. **Zoom and Pan**: Use your mouse wheel to zoom in and out. Click and drag to pan across different regions of the plot.
3. **Identify Clusters**: Look for areas where points are densely packed together. These could represent compounds with similar mechanisms of action or structural features.
4. **Note Outliers**: Pay attention to points that are far away from any cluster. These could be novel compounds with unique properties worth investigating.

#### Let your curiosity guide you in uncovering the next blockbuster drug!
- **High-Affinity Clusters**: Are there clusters of compounds with high binding affinity? These could be strong candidates for drug development.
- **Agonist/Antagonist Patterns**: Do you notice any trends related to the probability of being an agonist or antagonist? This could help in targeting specific pathways.
- **Anomalies and Novel Compounds**: Are there any points far away from any clusters? These could be novel scaffolds or unique mechanisms of action that are worth exploring.


In [ ]:
#@title 1. Data Setup for Interactive Platform
from tqdm.notebook import tqdm
import umap
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
df = pd.read_csv(os.path.join(result_path,'screening.csv'))

fps = []
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):
    fp = np.load(os.path.join(result_path,'interpretation_result',row['ID'],'fingerprint.npy'))
    fps.append(fp)

fps = np.stack(fps)
reducer = umap.UMAP()
embedding = reducer.fit_transform(fps)
df[['x','y']] = embedding



In [ ]:
#@title 2. Interactive Platform for Drug Discovery

import plotly.graph_objects as go

#@markdown Scaling factor will adjust the size of each dot. Adjust the figure width and height for optimal visual.
scaling_factor = 0.9 #@param {type:"number"}
figure_width = 1500 #@param {type:"integer"}
figure_height = 1000 #@param {type:"integer"}

df[['x','y']] = embedding
df = df.rename(columns={'predicted_agonist':'Agonist Probability',
                'predicted_antagonist':'Antagonist Probability',
                'predicted_nonbinder':'Non-binder Probability',
                'predicted_binding_affinity':'Binding Affinity'})

hover_data_list = ['ID','Binding Affinity']

def exponential_scale(value, base=2):
    return 4 + (base ** (value - 4))
def functional_effect_shape(cls):
    if cls == 'Agonist':
        return 'triangle-up'
    elif cls == 'Antagonist':
        return 'triangle-down'
    else:
        return 'circle'

df['Binding Affinity Size'] = exponential_scale(df['Binding Affinity'].clip(4,10)) * scaling_factor
if 'Agonist Probability' in df.columns and 'Antagonist Probability' in df.columns and 'Non-binder Probability' in df.columns:
    # Determine the shape based on the highest probability category
    df['Functional Effect Class'] = df[['Agonist Probability', 'Antagonist Probability', 'Non-binder Probability']].idxmax(axis=1)
    df['Functional Effect Class'] = df['Functional Effect Class'].apply(lambda x: x.replace(' Probability',''))
    df['Shape'] = df['Functional Effect Class'].apply(functional_effect_shape)
    hover_data_list += ['Agonist Probability', 'Antagonist Probability', 'Non-binder Probability']


hover_string = "ID: %{customdata[0]}<br>Binding Affinity: %{customdata[1]:.4f}"
if 'Shape' in df.columns:
    hover_string += "<br>Agonist Probability: %{customdata[2]:.4f}<br>Antagonist Probability: %{customdata[3]:.4f}<br>Non-binder Probability: %{customdata[4]:.4f}"

# Customize color bar
max_value = df['Binding Affinity Size'].max()
min_value = df['Binding Affinity Size'].min()
ori_max_value = df['Binding Affinity'].max()
ori_min_value = df['Binding Affinity'].min()


fig = go.Figure()

if 'Functional Effect Class' in df.columns:
    # Loop through each unique class to create a separate trace
    for cls in df['Functional Effect Class'].unique():
        subset_df = df[df['Functional Effect Class'] == cls]
        fig.add_trace(go.Scatter(
            x=subset_df['x'],
            y=subset_df['y'],
            mode='markers',
            marker=dict(
                size=subset_df['Binding Affinity Size'],
                color=subset_df['Binding Affinity Size'],
                colorscale=px.colors.sequential.Redor,
                colorbar=dict(
                    title="Binding Affinity Size",
                    tickvals=[min_value, max_value],
                    ticktext=[f"Affinity Min: {ori_min_value}", f"Affinity Max: {ori_max_value}"]
                ),
                symbol=subset_df['Functional Effect Class'].apply(lambda x: functional_effect_shape(x))
            ),
            name=cls,  # This name will appear in the legend
            customdata=subset_df[hover_data_list],
            hovertemplate=hover_string
        ))

    # Customize hover template
    fig.update_traces(
        hovertemplate=hover_string
    )

    fig.update_layout(
        legend=dict(
            x=0,  # Adjust as needed
            y=1.1,
            traceorder="normal",
            orientation="h",
            font=dict(
                size=15
            )

        ),
        width=figure_width,  # Width in pixels
        height=figure_height # Height in pixels
    )
else:
    fig.add_trace(go.Scatter(
            x=df['x'],
            y=df['y'],
            mode='markers',
            marker=dict(
                size=df['Binding Affinity Size'],
                color=df['Binding Affinity Size'],
                colorscale=px.colors.sequential.Redor,
                colorbar=dict(
                    title="Binding Affinity Size",
                    tickvals=[min_value, max_value],
                    ticktext=[f"Affinity Min: {ori_min_value}", f"Affinity Max: {ori_max_value}"]
                ),
            ),
            customdata=df[hover_data_list],
            name='',  # Set name to an empty string
            hovertemplate=hover_string
        ))

    # Customize hover template
    fig.update_traces(
        hovertemplate=hover_string
    )
fig.show()


# PSICHIC's Interpretation of Protein-Ligand Interaction

In [ ]:
#@title 2. Visualisation of PSICHIC Importance Scores
pair_id = "id_6oxw" #@param {type:"string"}
protein_path='' #@param {type:"string"}
ligand_path= '' #@param {type:"string"}

if not protein_path:
    protein_seq = df[df['ID'] == pair_id]['Protein'].item()
    print('protein sequence length:', len(protein_seq))
    if len(protein_seq) > 700:
        raise Exception('Protein sequence length is greater than 700, this will cause out-of-memory error. Please use separate tools to get in silico structures.')
residue_color = "Original" #@param ["Original", "Percentile Ranking"]
show_protein_mainchain = False #@param {type:"boolean"}
show_protein_surface = True #@param {type:"boolean"}
show_protein_sidechain = False #@param {type:"boolean"}


protein_interpret = os.path.join(result_path,'interpretation_result',pair_id,'protein.csv')
ligand_interpret = os.path.join(result_path,'interpretation_result',pair_id,'ligand.pkl')
protein_block, ligand_block = plot_visual(protein_interpret, ligand_interpret,
                                          protein_path=protein_path, ligand_path=ligand_path,
                                          residue_color_type=residue_color,
                                          show_protein_surface=show_protein_surface,
                                          show_protein_mainchain=show_protein_mainchain,
                                          show_protein_sidechain=show_protein_sidechain)

In [ ]:
#@title 3. OPTIONAL - Download Interpretation Results for PyMol
#@markdown If you are having issues downloading the result archive, try disabling your adblocker and run this cell again. If that fails click on the little folder icon to the left, navigate to file: `jobname.result.zip`, right-click and select \"Download\" (see [screenshot](https://pbs.twimg.com/media/E6wRW2lWUAEOuoe?format=jpg&name=small)).

#@title Do you agree? { run: "auto" }
download_result = True #@param {type:"boolean"}

if download_result:
    # files.download(f"{jobname}.result.zip")
    pass